In [25]:
# Importing all necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime

# Flight Delay Classification Model Building

This notebook outlines the code pull in more of the data that will later be used to create a classification model for flights being delayed.

### Data

The main dataset was pulled from [kaggle](https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018/code "2008-2019 Domestic Flight Data"). 

The data from this source was initialized in the prior notebook [Flight Delays Initial Data Pulling (1)]()

Additional data will need to be merged to our main dataframe, including:
* Weather conditions (also pulled from [kaggle](https://www.kaggle.com/sobhanmoosavi/us-weather-events "2016-2019 Adverse Airport Weather Conditions DataSet"))

#### Pulling in data from first notebook

In [28]:
df=pd.read_csv('/Users/mehikapatel/Flights_Project/Data/FlightsDataAfterNB1')

In [29]:
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['DATE','AIRLINE','FLIGHT_NUM','ORIGIN','DEST'],inplace=True)

*Drop the unnamed & AirportCode columns at the start & Make the DATE column a dt type*

In [34]:
df.drop(columns=['Unnamed: 0','AirportCode'],inplace=True)
df.DATE=pd.to_datetime(df['DATE'])

**This will create our target variable column. We decide to mark delays as those that are at least 20 minutes late in arrival.**

In [35]:
df['target'] = df['ARR_DELAY']>=20

**Now time to pull in historic weather data.**

In [36]:
#pull weather data into dataframe
weather_data=pd.read_csv('/Users/mehikapatel/Flights_Project/Data/WeatherEventsData.csv')

In [37]:
#delete unecessary columns from weather df
weather_data.drop(columns=['EventId','County','ZipCode','State','EndTime(UTC)','TimeZone','LocationLat','LocationLng','AirportCode'],inplace=True)
weather_data.rename({'City':'municipality1'},axis=1,inplace=True)

First we have to make the times compatible with our main df.

In [38]:
#look at weather data
weather_data['DATE']=pd.to_datetime(weather_data['StartTime(UTC)']).dt.date
weather_data['DATE']=pd.to_datetime(weather_data['DATE'])
weather_data.drop(columns=['StartTime(UTC)'],inplace=True)

In [39]:
# weather_data['StartHour']=weather_data['StartTime(UTC)'].dt.hour
# weather_data['EndHour']=weather_data['EndTime(UTC)'].dt.hour
weather_data.drop_duplicates(inplace=True)

In [40]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2502240 entries, 0 to 6273468
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   Type           object        
 1   Severity       object        
 2   municipality1  object        
 3   DATE           datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 95.5+ MB


In [41]:
#merge for municipality1
df=df.merge(weather_data,how='left',on=['DATE','municipality1'])

In [42]:
#need to remove some duplicates because of duplicates found. We originally had 12,477,222 data points. After 
# removing the original duplicates, we had 18426666. After removing our duplicates, we have ridden some of the data points.
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['DATE','AIRLINE','FLIGHT_NUM','ORIGIN','DEST'],inplace=True)

In [47]:
df.head()
# weather_data.head()

,DATE,AIRLINE,FLIGHT_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,...,dest_type,dest_lat,dest_lon,municipality2,holiday_szn,DEP_HOUR,ARR_HOUR,target,origin_weather,origin_severity
0,2016-01-01,Delta Airlines,1248,DTW,LAX,1935,1935.0,0.0,2144,-24.0,...,large,33.942501,-118.407997,Los Angeles,True,19,21,False,Snow,Light
2,2016-01-01,Delta Airlines,1251,ATL,GRR,2125,2130.0,5.0,2321,-2.0,...,medium,42.880798,-85.522797,Grand Rapids,True,21,23,False,NaN,NaN
3,2016-01-01,Delta Airlines,1255,SLC,ATL,1656,1700.0,4.0,2229,-16.0,...,large,33.636700,-84.428101,Atlanta,True,16,22,False,Fog,Severe
5,2016-01-01,Delta Airlines,1257,ATL,BNA,1233,1356.0,83.0,1239,83.0,...,large,36.124500,-86.678200,Nashville,True,12,12,True,NaN,NaN
6,2016-01-01,Delta Airlines,1257,BNA,ATL,1320,1446.0,86.0,1530,74.0,...,large,33.636700,-84.428101,Atlanta,True,13,15,True,NaN,NaN


In [45]:
#now remove hour data from weather_data  & rename municipality1 to municipality2
weather_data.rename({'municipality1':'municipality2'},axis=1,inplace=True)

In [46]:
#now rename in main df the type and severity columns
df.rename({'Type':'origin_weather', 'Severity':'origin_severity'},axis=1,inplace=True)

In [48]:
#merge for destination weather
df=df.merge(weather_data,how='left',on=['DATE','municipality2'])

In [53]:
#need to remove some duplicates because of duplicates found. We originally had 12,477,222 data points. After 
# removing the duplicates on all columns, we had _____. After removing our duplicates, we have ridden some of the data points.
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['DATE','AIRLINE','FLIGHT_NUM','ORIGIN','DEST'],inplace=True)

In [57]:
#Check number of duplicates
df.head()

,DATE,AIRLINE,FLIGHT_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,...,dest_lon,municipality2,holiday_szn,DEP_HOUR,ARR_HOUR,target,origin_weather,origin_severity,dest_weather,dest_severity
0,2016-01-01,Delta Airlines,1248,DTW,LAX,1935,1935.0,0.0,2144,-24.0,...,-118.407997,Los Angeles,True,19,21,False,Snow,Light,Fog,Moderate
1,2016-01-01,Delta Airlines,1251,ATL,GRR,2125,2130.0,5.0,2321,-2.0,...,-85.522797,Grand Rapids,True,21,23,False,NaN,NaN,Snow,Light
2,2016-01-01,Delta Airlines,1255,SLC,ATL,1656,1700.0,4.0,2229,-16.0,...,-84.428101,Atlanta,True,16,22,False,Fog,Severe,NaN,NaN
3,2016-01-01,Delta Airlines,1257,ATL,BNA,1233,1356.0,83.0,1239,83.0,...,-86.678200,Nashville,True,12,12,True,NaN,NaN,NaN,NaN
4,2016-01-01,Delta Airlines,1257,BNA,ATL,1320,1446.0,86.0,1530,74.0,...,-84.428101,Atlanta,True,13,15,True,NaN,NaN,NaN,NaN


In [55]:
#rename new columns
df.rename({'Type':'dest_weather', 'Severity':'dest_severity'},axis=1,inplace=True)

In [ ]:
#save to csv as final DF
df.to_csv('/Users/mehikapatel/Flights_Project/FinalFlightsData.csv')

Make a subset for just 2018 flights.

In [8]:
#only 2018
year_2018=[2018]
df['is_year']=df.YEAR.isin(year_2018)
index_names = df[ df['is_year'] == False ].index
data_2018 = df.drop(index_names)
df.drop(columns=['is_year'],inplace=True)

In [17]:
data_2018=data_2018.drop(columns=['DATE','FLIGHT_NUM','CRS_DEP_TIME','DEP_TIME','DEP_DELAY','CRS_ARR_TIME','ARR_DELAY',
                                 'CANCELLED','origin_lat','origin_lon','dest_lat','dest_lon','YEAR','is_year'])

In [18]:
sample_df = data_2018.sample(10000,random_state=42)

In [19]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 12265898 to 9587694
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AIRLINE           10000 non-null  object 
 1   ORIGIN            10000 non-null  object 
 2   DEST              10000 non-null  object 
 3   CRS_ELAPSED_TIME  10000 non-null  float64
 4   DISTANCE          10000 non-null  float64
 5   MONTH             10000 non-null  int64  
 6   DAYOFWEEK         10000 non-null  int64  
 7   origin_type       9960 non-null   object 
 8   municipality_x    9960 non-null   object 
 9   dest_type         9938 non-null   object 
 10  municipality2     9938 non-null   object 
 11  holiday_szn       10000 non-null  bool   
 12  DEP_HOUR          10000 non-null  int64  
 13  ARR_HOUR          10000 non-null  int64  
 14  target            10000 non-null  bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 1.1+ MB


**There are about 12 million data points to work with, so we pull a random sample of 10,000 points for initial 
plotting purposes**

Plot Features from 2018 data for initial feature recognition.